# **Import Libraries**

In [1]:
!pip install transformers datasets sentencepiece sacremoses
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
%env TORCH_USE_CUDA_DSA=1
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from datetime import datetime
from transformers import XLNetTokenizer
from torch.utils.data import DataLoader


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 72.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.1 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=4ef9ff931be54283cb7c126a93b777322683c9e39545d28e3fa0709c90131

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Build Model**

In [3]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_k, d_model, n_heads, max_len, causal=False):
    super().__init__()

    self.d_k = d_k
    self.n_heads = n_heads

    self.key = nn.Linear(d_model, d_k * n_heads)
    self.query = nn.Linear(d_model, d_k * n_heads)
    self.value = nn.Linear(d_model, d_k * n_heads)

    self.fc = nn.Linear(d_k * n_heads, d_model)

    self.causal = causal
    if causal:
      cm = torch.tril(torch.ones(max_len, max_len))
      self.register_buffer(
          "causal_mask",
          cm.view(1, 1, max_len, max_len)
      )

  def forward(self, q, k, v, pad_mask=None):
    q = self.query(q) # N x T x (hd_k)
    k = self.key(k)
    v = self.value(v)

    N = q.shape[0]
    T_output = q.shape[1]
    T_input = k.shape[1]

    # (N, T, h, d_k) -> (N, h, T, d_k)
    q = q.view(N, T_output, self.n_heads, self.d_k).transpose(1, 2)
    k = k.view(N, T_input, self.n_heads, self.d_k).transpose(1, 2)
    v = v.view(N, T_input, self.n_heads, self.d_k).transpose(1, 2)

    # (N, h, T, d_k) x (N, h, d_k, T) --> (N, h, T, T)
    attn_scores = q @ k.transpose(-2, -1) / math.sqrt(self.d_k)
    if pad_mask is not None:
      attn_scores = attn_scores.masked_fill(
          pad_mask[:, None, None, :] == 0, float('-inf'))
    if self.causal:
      attn_scores = attn_scores.masked_fill(
          self.causal_mask[:, :, :T_output, :T_input] == 0, float('-inf'))
    attn_weights = F.softmax(attn_scores, dim=-1)

    # compute attention-weighted values: (N, h, T, T) x (N, h, T, d_k) --> (N, h, T, d_k)
    A = attn_weights @ v

    A = A.transpose(1, 2) # (N, T, h, d_k)
    A = A.contiguous().view(N, T_output, self.d_k * self.n_heads)

    # projection
    return self.fc(A)

In [4]:
class EncoderBlock(nn.Module):
  def __init__(self, d_k, d_model, n_heads, max_len, dropout_prob=0.1):
    super().__init__()

    self.ln1 = nn.LayerNorm(d_model)
    self.ln2 = nn.LayerNorm(d_model)
    self.mha = MultiHeadAttention(d_k, d_model, n_heads, max_len, causal=False)
    self.ann = nn.Sequential(
        nn.Linear(d_model, d_model * 4),
        nn.GELU(),
        nn.Linear(d_model * 4, d_model),
        nn.Dropout(dropout_prob),
    )
    self.dropout = nn.Dropout(p=dropout_prob)

  def forward(self, x, pad_mask=None):
    x = self.ln1(x + self.mha(x, x, x, pad_mask))
    x = self.ln2(x + self.ann(x))
    x = self.dropout(x)
    return x

In [5]:
class DecoderBlock(nn.Module):
  def __init__(self, d_k, d_model, n_heads, max_len, dropout_prob=0.1):
    super().__init__()

    self.ln1 = nn.LayerNorm(d_model)
    self.ln2 = nn.LayerNorm(d_model)
    self.ln3 = nn.LayerNorm(d_model)
    self.mha1 = MultiHeadAttention(d_k, d_model, n_heads, max_len, causal=True)
    self.mha2 = MultiHeadAttention(d_k, d_model, n_heads, max_len, causal=False)
    self.ann = nn.Sequential(
        nn.Linear(d_model, d_model * 4),
        nn.GELU(),
        nn.Linear(d_model * 4, d_model),
        nn.Dropout(dropout_prob),
    )
    self.dropout = nn.Dropout(p=dropout_prob)

  def forward(self, enc_output, dec_input, enc_mask=None, dec_mask=None):
    x = self.ln1(dec_input + self.mha1(dec_input, dec_input, dec_input, dec_mask))

    #multi-head attention include encoder output
    x = self.ln2(x + self.mha2(x, enc_output, enc_output, enc_mask))

    x = self.ln3(x + self.ann(x))
    x = self.dropout(x)
    return x

In [6]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_len=2048, dropout_prob=0.1):
    super().__init__()
    self.dropout = nn.Dropout(p=dropout_prob)

    position = torch.arange(max_len).unsqueeze(1)
    exp_term = torch.arange(0, d_model, 2)
    div_term = torch.exp(exp_term * (-math.log(10000.0) / d_model))
    pe = torch.zeros(1, max_len, d_model)
    pe[0, :, 0::2] = torch.sin(position * div_term)
    pe[0, :, 1::2] = torch.cos(position * div_term)
    self.register_buffer('pe', pe)

  def forward(self, x):
    x = x + self.pe[:, :x.size(1), :]
    return self.dropout(x)

In [7]:
class Encoder(nn.Module):
  def __init__(self,
               vocab_size,
               max_len,
               d_k,
               d_model,
               n_heads,
               n_layers,
              #  n_classes,
               dropout_prob):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = PositionalEncoding(d_model, max_len, dropout_prob)
    transformer_blocks = [
        EncoderBlock(
            d_k,
            d_model,
            n_heads,
            max_len,
            dropout_prob) for _ in range(n_layers)]
    self.transformer_blocks = nn.Sequential(*transformer_blocks)
    self.ln = nn.LayerNorm(d_model)

  def forward(self, x, pad_mask=None):
    x = self.embedding(x)
    x = self.pos_encoding(x)
    for block in self.transformer_blocks:
      x = block(x, pad_mask)

    x = self.ln(x)
    return x

In [8]:
class Decoder(nn.Module):
  def __init__(self,
               vocab_size,
               max_len,
               d_k,
               d_model,
               n_heads,
               n_layers,
               dropout_prob):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = PositionalEncoding(d_model, max_len, dropout_prob)
    transformer_blocks = [
        DecoderBlock(
            d_k,
            d_model,
            n_heads,
            max_len,
            dropout_prob) for _ in range(n_layers)]
    self.transformer_blocks = nn.Sequential(*transformer_blocks)
    self.ln = nn.LayerNorm(d_model)
    self.fc = nn.Linear(d_model, vocab_size)

  def forward(self, enc_output, dec_input, enc_mask=None, dec_mask=None):
    x = self.embedding(dec_input)
    x = self.pos_encoding(x)
    for block in self.transformer_blocks:
      x = block(enc_output, x, enc_mask, dec_mask)
    x = self.ln(x)
    x = self.fc(x)
    return x

In [9]:
class Transformer(nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, enc_input, dec_input, enc_mask, dec_mask):
    enc_output = self.encoder(enc_input, enc_mask)
    dec_output = self.decoder(enc_output, dec_input, enc_mask, dec_mask)
    return dec_output

In [10]:
class ScheduledOptimizer():
    def __init__(self, optimizer, init_lr, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.init_lr = init_lr
        self.d_model = d_model
        self.n_warmup_steps = n_warmup_steps
        self.n_steps = 0


    def step_and_update_lr(self):
        self._update_learning_rate()
        self._optimizer.step()


    def zero_grad(self):
        self._optimizer.zero_grad()


    def _get_lr_scale(self):
        d_model = self.d_model
        n_steps, n_warmup_steps = self.n_steps, self.n_warmup_steps
        return (d_model ** -0.5) * min(n_steps ** (-0.5), n_steps * n_warmup_steps ** (-1.5))

    def state_dict(self):
        optimizer_state_dict = {
            'init_lr':self.init_lr,
            'd_model':self.d_model,
            'n_warmup_steps':self.n_warmup_steps,
            'n_steps':self.n_steps,
            '_optimizer':self._optimizer.state_dict(),
        }

        return optimizer_state_dict

    def load_state_dict(self, state_dict):
        self.init_lr = state_dict['init_lr']
        self.d_model = state_dict['d_model']
        self.n_warmup_steps = state_dict['n_warmup_steps']
        self.n_steps = state_dict['n_steps']

        self._optimizer.load_state_dict(state_dict['_optimizer'])

    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_steps += 1
        lr = self.init_lr * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr

# **Load and Preprocess Data**

In [11]:
pt_raw_dataset = load_dataset('csv', data_files='/content/gdrive/MyDrive/Transformer/data/pretrain_dataset.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-aba1d4e042fd98d1/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
pt_raw_dataset

DatasetDict({
    train: Dataset({
        features: ['en'],
        num_rows: 372854
    })
})

In [13]:
raw_dataset = load_dataset('csv', data_files='/content/gdrive/MyDrive/Transformer/data/dataset.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-056c76c9d490d8ce/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'vi'],
        num_rows: 941338
    })
})

### *Split Data (70/15/15)**

In [15]:
#train
pt_dataset = pt_raw_dataset['train'].train_test_split(test_size=0.3, train_size=0.7, shuffle=True, seed=42)
pt_train_dataset = pt_dataset['train']
#val
pt_val_test_dataset = pt_dataset['test']

In [16]:
#train
dataset = raw_dataset['train'].train_test_split(test_size=0.3, train_size=0.7, shuffle=True, seed=42)
train_dataset = dataset['train']
#val
val_test_dataset = dataset['test']
val_test_split = val_test_dataset.train_test_split(test_size=0.5, shuffle=True, seed=42)
val_dataset = val_test_split['train']
#test
test_dataset = val_test_split['test']

In [17]:
pt_tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
# en_sentence = pt_train_dataset[5]["en"]
# targets = pt_tokenizer(text_target=en_sentence)
# pt_tokenizer.convert_ids_to_tokens(targets['input_ids'])

In [18]:
max_input_length = 128
max_target_length = 128

def pt_preprocess_function(batch):
    model_inputs = pt_tokenizer(
        batch['en'], max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    labels = pt_tokenizer(
        text_target=batch['en'], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [19]:
pt_tokenized_datasets = pt_dataset.map(
    pt_preprocess_function,
    batched=True,
    remove_columns=pt_dataset["train"].column_names,
)

Map:   0%|          | 0/260997 [00:00<?, ? examples/s]

Map:   0%|          | 0/111857 [00:00<?, ? examples/s]

In [20]:
pt_tokenizer.add_special_tokens({"cls_token": "<s>"})
pt_tokenizer.vocab_size

32000

In [21]:
pt_data_collator = DataCollatorForSeq2Seq(pt_tokenizer)

In [22]:
pt_train_loader = DataLoader(
    pt_tokenized_datasets["train"],
    shuffle=True,
    batch_size=32,
    collate_fn=pt_data_collator
)
pt_valid_loader = DataLoader(
    pt_tokenized_datasets["test"],
    batch_size=32,
    collate_fn=pt_data_collator
)

In [23]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [24]:
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-vi', vocab_size=53685, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True)

In [25]:
#example
en_sentence = train_dataset[2]["en"]
vi_sentence = train_dataset[2]["vi"]

inputs = tokenizer(en_sentence)
targets = tokenizer(text_target=vi_sentence)

tokenizer.convert_ids_to_tokens(targets['input_ids'])

['▁anh',
 '▁che',
 '▁giấu',
 '▁sự',
 '▁thật',
 '▁rằng',
 '▁anh',
 '▁đã',
 '▁ly',
 '▁dị',
 '▁vợ',
 '</s>']

In [26]:
vi_sentence

'anh che giấu sự thật rằng anh đã ly dị vợ'

In [27]:
max_input_length = 128
max_target_length = 128

def preprocess_function(batch):
    model_inputs = tokenizer(
        batch['en'], max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    labels = tokenizer(
        text_target=batch['vi'], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [28]:
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/658936 [00:00<?, ? examples/s]

Map:   0%|          | 0/282402 [00:00<?, ? examples/s]

In [29]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 658936
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 282402
    })
})

In [30]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [31]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(0, 5)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [32]:
tokenizer.all_special_ids

[0, 1, 53684]

In [33]:

train_loader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
valid_loader = DataLoader(
    tokenized_datasets["test"],
    batch_size=32,
    collate_fn=data_collator
)

In [34]:
tokenizer.add_special_tokens({"cls_token": "<s>"})


1

In [35]:
tokenizer.vocab_size

53685

# **Config Model**

In [36]:
encoder = Encoder(vocab_size=53685 + 1,
                  max_len=512,
                  d_k=16,
                  d_model=512,
                  n_heads=8,
                  n_layers=6,
                  dropout_prob=0.1)
decoder = Decoder(vocab_size=53685 + 1,
                  max_len=512,
                  d_k=16,
                  d_model=512,
                  n_heads=8,
                  n_layers=6,
                  dropout_prob=0.1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
encoder.to(device)
decoder.to(device)
transformer = Transformer(encoder, decoder)

cuda:0


### *Load old model and optimizer from previous trainning*

In [37]:
# Load old model and optimization
transformer.load_state_dict(torch.load('/content/gdrive/MyDrive/Transformer/data/transformer_model.pth'))
optimizer = torch.optim.Adam(transformer.parameters(), betas=(0.9, 0.98), eps=1e-09)
optimizer = ScheduledOptimizer(optimizer,0.2, 64, 4000)
optimizer.load_state_dict(torch.load('/content/gdrive/MyDrive/Transformer/data/optimizer.pth'))

In [38]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=-100)

### *Train*

In [39]:
def pretrain_train(model, criterion, optimizer, train_loader, valid_loader, epochs):
  train_losses = np.zeros(epochs)
  test_losses = np.zeros(epochs)

  for it in range(epochs):
    model.train()
    t0 = datetime.now()
    train_loss = []
    for batch in train_loader:
      batch = {k: v.to(device) for k, v in batch.items()}

      optimizer.zero_grad()

      enc_input = batch['input_ids']
      enc_mask = batch['attention_mask']
      targets = batch['labels']

      dec_input = targets.clone().detach()
      dec_input = torch.roll(dec_input, shifts=1, dims=1)
      dec_input[:, 0] = 32_000

      dec_input = dec_input.masked_fill(
          dec_input == -100, pt_tokenizer.pad_token_id)

      dec_mask = torch.ones_like(dec_input)
      dec_mask = dec_mask.masked_fill(dec_input == pt_tokenizer.pad_token_id, 0)

      outputs = model(enc_input, dec_input, enc_mask, dec_mask)
      loss = criterion(outputs.transpose(2, 1), targets)

      loss.backward()
      optimizer.step_and_update_lr()
      train_loss.append(loss.item())

    train_loss = np.mean(train_loss)

    model.eval()
    test_loss = []
    for batch in valid_loader:
      batch = {k: v.to(device) for k, v in batch.items()}

      enc_input = batch['input_ids']
      enc_mask = batch['attention_mask']
      targets = batch['labels']

      dec_input = targets.clone().detach()
      dec_input = torch.roll(dec_input, shifts=1, dims=1)
      dec_input[:, 0] = 32_000

      dec_input = dec_input.masked_fill(dec_input == -100, pt_tokenizer.pad_token_id)

      dec_mask = torch.ones_like(dec_input)
      dec_mask = dec_mask.masked_fill(dec_input == pt_tokenizer.pad_token_id, 0)

      outputs = model(enc_input, dec_input, enc_mask, dec_mask)
      loss = criterion(outputs.transpose(2, 1), targets)
      test_loss.append(loss.item())
    test_loss = np.mean(test_loss)

    # Save losses
    train_losses[it] = train_loss
    test_losses[it] = test_loss

    dt = datetime.now() - t0

    print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
      Test Loss: {test_loss:.4f}, Duration: {dt}')
    # path to save
    model_path = "/content/gdrive/MyDrive/Transformer/data/transformer_model.pth"
    optimizer_path = "/content/gdrive/MyDrive/Transformer/data/optimizer.pth"

    # save model
    torch.save(model.state_dict(), model_path)
    torch.save(optimizer.state_dict(), optimizer_path)
  return train_losses, test_losses

In [40]:
train_losses, test_losses = pretrain_train(transformer, criterion, optimizer, pt_train_loader, pt_valid_loader, epochs=12)

Epoch 1/12, Train Loss: 2.4525,       Test Loss: 0.6228, Duration: 0:35:33.326281
Epoch 2/12, Train Loss: 0.8614,       Test Loss: 0.3293, Duration: 0:35:47.323107
Epoch 3/12, Train Loss: 0.5865,       Test Loss: 0.2384, Duration: 0:35:48.742753
Epoch 4/12, Train Loss: 0.4665,       Test Loss: 0.1994, Duration: 0:35:51.591239
Epoch 5/12, Train Loss: 0.4009,       Test Loss: 0.1839, Duration: 0:35:50.931504
Epoch 6/12, Train Loss: 0.3584,       Test Loss: 0.1714, Duration: 0:35:55.316295


KeyboardInterrupt: ignored

In [41]:
def finetune_train(model, criterion, optimizer, train_loader, valid_loader, epochs):
  train_losses = np.zeros(epochs)
  test_losses = np.zeros(epochs)

  for it in range(epochs):
    model.train()
    t0 = datetime.now()
    train_loss = []
    for batch in train_loader:
      batch = {k: v.to(device) for k, v in batch.items()}

      optimizer.zero_grad()

      enc_input = batch['input_ids']

      #L1
      pt_targets = enc_input[:, 1:].contiguous().view(enc_input.size(0), -1)

      pt_dec_input = pt_targets.clone().detach()
      pt_dec_input = torch.roll(pt_dec_input, shifts=1, dims=1)
      pt_dec_input[:, 0] = 32_000

      pt_dec_input = pt_dec_input.masked_fill(
          pt_dec_input == -100, pt_tokenizer.pad_token_id)

      pt_dec_mask = torch.ones_like(pt_dec_input)
      pt_dec_mask = pt_dec_mask.masked_fill(pt_dec_input == pt_tokenizer.pad_token_id, 0)

      pt_outputs = model(enc_input, pt_dec_input, None, pt_dec_mask)
      loss_1 = criterion(pt_outputs.transpose(2, 1), pt_targets)
      # print("loss 1: ", loss_1)
      #L2
      enc_mask = batch['attention_mask']
      targets = batch['labels']

      dec_input = targets.clone().detach()
      dec_input = torch.roll(dec_input, shifts=1, dims=1)
      dec_input[:, 0] = 53_685

      dec_input = dec_input.masked_fill(
          dec_input == -100, tokenizer.pad_token_id)

      dec_mask = torch.ones_like(dec_input)
      dec_mask = dec_mask.masked_fill(dec_input == tokenizer.pad_token_id, 0)

      outputs = model(enc_input, dec_input, enc_mask, dec_mask)
      loss_2 = criterion(outputs.transpose(2, 1), targets)
      # print("loss 2: ", loss_2)

      loss_3 = loss_2 + 0.25*loss_1
      loss_3.backward()
      optimizer.step_and_update_lr()
      train_loss.append(loss_3.item())

    train_loss = np.mean(train_loss)

    model.eval()
    test_loss = []
    for batch in valid_loader:
      batch = {k: v.to(device) for k, v in batch.items()}

      enc_input = batch['input_ids']
      #L1
      pt_targets = enc_input[:, 1:].contiguous().view(enc_input.size(0), -1)

      pt_dec_input = pt_targets.clone().detach()
      pt_dec_input = torch.roll(pt_dec_input, shifts=1, dims=1)
      pt_dec_input[:, 0] = 32_000

      pt_dec_input = pt_dec_input.masked_fill(
          pt_dec_input == -100, pt_tokenizer.pad_token_id)

      pt_dec_mask = torch.ones_like(pt_dec_input)
      pt_dec_mask = pt_dec_mask.masked_fill(pt_dec_input == pt_tokenizer.pad_token_id, 0)

      pt_outputs = model(enc_input, pt_dec_input, None, pt_dec_mask)
      loss_1 = criterion(pt_outputs.transpose(2, 1), pt_targets)
      # print("loss 1: ", loss_1)
      #L2
      enc_mask = batch['attention_mask']
      targets = batch['labels']

      dec_input = targets.clone().detach()
      dec_input = torch.roll(dec_input, shifts=1, dims=1)
      dec_input[:, 0] = 53_685

      dec_input = dec_input.masked_fill(
          dec_input == -100, tokenizer.pad_token_id)

      dec_mask = torch.ones_like(dec_input)
      dec_mask = dec_mask.masked_fill(dec_input == tokenizer.pad_token_id, 0)

      outputs = model(enc_input, dec_input, enc_mask, dec_mask)
      loss_2 = criterion(outputs.transpose(2, 1), targets)
      # print("loss 2: ", loss_2)

      loss_3 = loss_2 + 0.25*loss_1
      test_loss.append(loss_3.item())

    test_loss = np.mean(test_loss)
    # Save losses
    train_losses[it] = train_loss
    test_losses[it] = test_loss

    dt = datetime.now() - t0

    print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
      Test Loss: {test_loss:.4f}, Duration: {dt}')
    # path to save
    model_path = "/content/gdrive/MyDrive/Transformer/data/transformer_model.pth"
    optimizer_path = "/content/gdrive/MyDrive/Transformer/data/optimizer.pth"

    # save model
    torch.save(model.state_dict(), model_path)
    torch.save(optimizer.state_dict(), optimizer_path)
  return train_losses, test_losses

In [ ]:
train_losses, test_losses = finetune_train(transformer, criterion, optimizer, train_loader, valid_loader, epochs=12)

Epoch 1/12, Train Loss: 2.0901,       Test Loss: 1.8136, Duration: 2:14:25.057243
Epoch 2/12, Train Loss: 1.7488,       Test Loss: 1.7768, Duration: 2:14:23.665082
Epoch 3/12, Train Loss: 1.7016,       Test Loss: 1.7582, Duration: 2:14:11.621383
Epoch 4/12, Train Loss: 1.6748,       Test Loss: 1.7522, Duration: 2:13:15.474345
Epoch 5/12, Train Loss: 1.6583,       Test Loss: 1.7456, Duration: 2:12:31.619857
Epoch 6/12, Train Loss: 1.6445,       Test Loss: 1.7414, Duration: 2:12:14.596088
Epoch 7/12, Train Loss: 1.6329,       Test Loss: 1.7387, Duration: 2:11:59.351190
Epoch 8/12, Train Loss: 1.6239,       Test Loss: 1.7313, Duration: 2:13:26.548541
Epoch 9/12, Train Loss: 1.6156,       Test Loss: 1.7321, Duration: 2:13:19.466350


# **Predict**

In [ ]:
def translate(input_sentence, transformer):
  enc_input = tokenizer(input_sentence, return_tensors='pt').to(device)
  enc_output = transformer.encoder(enc_input['input_ids'], enc_input['attention_mask'])

  dec_input_ids = torch.tensor([[53_685]], device=device)
  dec_attn_mask = torch.ones_like(dec_input_ids, device=device)

  for _ in range(32):
    dec_output = transformer.decoder(
        enc_output,
        dec_input_ids,
        enc_input['attention_mask'],
        dec_attn_mask,
    )

    prediction_id = torch.argmax(dec_output[:, -1, :], axis=-1)
    dec_input_ids = torch.hstack((dec_input_ids, prediction_id.view(1, 1)))
    dec_attn_mask = torch.ones_like(dec_input_ids)

    if prediction_id == 0:
      break

  translation = tokenizer.decode(dec_input_ids[0, 1:-1])
  return translation

In [ ]:
translate("Why are people so pressed about the clapping, for that's the last thing you should think about in this situation.",transformer)

'tại sao mọi người lại áp đặt về tiếng vỗ tay, vì đó là điều cuối cùng bạn nên nghĩ về trong tình huống này'

In [ ]:
val_df = pd.DataFrame(val_dataset)
test_df = pd.DataFrame(test_dataset)

In [ ]:
val_df

,en,vi
0,Try to eat whole grains over refined grains .,Bạn nên ăn ngũ cốc nguyên hạt thay cho ngũ cốc...
1,It 's also worth noting that although the idea...,Nó sẽ không xứng đáng mặc dù ý kiến dường như ...
2,Dampening your skin is a great way to lower yo...,Làm mát da là một cách tuyệt vời để hạ nhiệt đ...
3,I didn't know why I wasn't supposed to go to t...,tôi không biết tại sao tôi không nên đến một p...
4,Could one have been mixed up for the other ?,Có khi nào hai câu chuyện đã bị nhầm lẫn với n...
...,...,...
141196,Now you can browse and stream movies and telev...,Bây giờ bạn có thể duyệt tìm và xem trực tuyến...
141197,Clear your mind of all the details you 're wor...,Dọn dẹp tâm trí khỏi những điều bạn đang lo lắ...
141198,"Instead , good sources of calcium for nursing ...","Thay vào đó , những thực phẩm giàu canxi dành ..."
141199,Women age faster than men,phụ nữ tuổi nhanh hơn nam giới


In [ ]:
# Giới hạn số dòng dịch thành 5 dòng đầu tiên
df_subset = val_df[:5000]

# Áp dụng hàm dịch vào DataFrame con
df_subset['vi_candidate'] = df_subset['en'].apply(lambda x: translate(x, transformer))


<ipython-input-43-25444cbee703>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['vi_candidate'] = df_subset['en'].apply(lambda x: translate(x, transformer))


In [ ]:
df_subset.to_csv('/content/gdrive/MyDrive/Transformer/data/val_candidate.csv', index=False)

In [ ]:
test_df['vi_candidate'] = test_df['en'].apply(lambda x: translate(x, transformer))
